In [1]:
%autosave 0
import torch
import torch.nn as nn
import transformers
from transformers import BertModel, BertForMaskedLM
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from string import punctuation
torch.manual_seed(1)
import random; random.seed(1)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import metrics
torch.random.manual_seed(1)
import matplotlib.pyplot as mat
import pandas as pd

Autosave disabled


In [2]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
tokenizer("S&P's")

{'input_ids': [101, 1055, 1004, 1052, 1005, 1055, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [7]:
#Corpus
file = open('/Users/sarkaajsingh/Desktop/SA_data_body_06',encoding="ISO-8859-1")
file=file.readlines()
corpus=file

In [8]:
len(corpus)

123227

In [9]:
corpus[0]

'Nikkei down pct hits lowest point in over years\n'

In [10]:
#Tokenize and lower casing
def preprocess_sentence(sentence):
  return tokenizer.tokenize(sentence)

sentences=[]
sentences1 = [preprocess_sentence(sent) for sent in corpus]
for i in range(len(sentences1)):
    sentences.append(sentences1[i][0:len(sentences1[i])])

In [11]:
len(sentences)

123227

In [12]:
print(sentences[0])

['nik', '##kei', 'down', 'pc', '##t', 'hits', 'lowest', 'point', 'in', 'over', 'years']


### Will return all with [0, 0, 1] for unlabelled data

In [13]:
#Labelling data with a response
y1=[]; y=[]
for i in range(len(sentences1)):
    y1.append(sentences1[i][len(sentences1[i])-1])

for i in y1:
    if i =='positive':
        i=[1,0,0]
    elif i=='neutral':
        i=[0,1,0]
    else: i=[0,0,1]
    y.append(i)

In [14]:
len(y)

123227

In [15]:
#Converting tokens to ids. Note that [CLS] [SEP] are included into the max length.
x=[tokenizer.encode_plus(sent, max_length=126+2,add_special_tokens=True,pad_to_max_length=True,
                         return_attention_mask=True,return_token_type_ids=False, truncation=True) for sent in sentences]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
x_input = [i['input_ids'] for i in x]
x_mask = [i['attention_mask'] for i in x]

In [17]:
(x_input[0])

[101,
 23205,
 29501,
 2091,
 7473,
 2102,
 4978,
 7290,
 2391,
 1999,
 2058,
 2086,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [18]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased',return_dict=False)
        self.l1 = nn.Linear(self.bert.config.hidden_size,3)
        self.prob=nn.Softmax(dim=1)
    
    # B = batch_size, L = sentence_length, D = embed_dim, H = hidden_size
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask) #Inputs () outputs () 
        out=self.l1(pooled_output) #Inputs (768) outputs (3)
        out=self.prob(out)
        return out

In [19]:
# Initialize a model
model = NeuralNet()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# Loading model (Must run all code before # Define an optimizer)
model.load_state_dict(torch.load('/Users/sarkaajsingh/Desktop/model_BERT_final_07',map_location=torch.device('cpu')))
model.eval()

NeuralNet(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [21]:
#Create a loader for loading data in desired format to be inputted into model
test_data = TensorDataset(torch.LongTensor(x_input), torch.LongTensor(x_mask), torch.LongTensor(y))
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [ ]:
# Printing loader information (ignore Batch_Labels here)
counter = 0
for batched_x_input, batched_x_mask, batched_y in test_loader:
  print(f"Iteration {counter}")
  print("Batched Input:")
  print(batched_x_input)
  print("Batched Mask:")
  print(batched_x_mask)
  print("Batched Labels:")
  print(batched_y)
  print("")
  counter += 1

In [22]:
  pred=[]; true=[]; results=[]; count=0
  for test_instance, test_mask, labels in test_loader:
    # Run a forward pass
    batch_size_model=1
    outputs = model.forward(test_instance, test_mask)
    results.append(outputs.detach().tolist()[0])
    #print(labels)
    #print(outputs)
    #Compute metrics
    pred_temp = [0,0,0]
    pred_temp[np.argmax((outputs.detach().tolist())[0])]=1
    pred.append(pred_temp)
    true_temp = labels.tolist()[0]
    true.append(true_temp)
    accuracy = metrics.accuracy_score(true,pred)*100
    f1=metrics.f1_score(true,pred,average="macro")*100
    count=count+1
    if count % 100 == 0 : print(count)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to contro

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [23]:
results[0]

[0.0030482930596917868, 0.009668192826211452, 0.98728346824646]

In [24]:
df=pd.DataFrame(results)
df.to_csv('/Users/sarkaajsingh/Desktop/results_06')

# The rest of the code is only for when you have labelled data

In [ ]:
classes = ['Positive', 'Neutral', 'Negative']
true1=[classes[np.argmax(i)] for i in true]
pred1=[classes[np.argmax(i)] for i in pred]

In [ ]:
print('Accuracy:', accuracy)
print('F1 Score:', f1)
print(metrics.classification_report(true1,pred1))
metrics.confusion_matrix(true1,pred1) #row is "true" and column is "predicted"
#row 1 is negative, row 2 is neutral, row 3 is positive

In [ ]:
np.sum(y,axis=0)

In [ ]:
print('Naive prediction accuracy:',max(np.sum(y,axis=0))/len(corpus)*100)